In [1]:
from regime_module import *
from statsmodels.iolib.summary2 import summary_col
import csv
import os

In [14]:
data_dir = os.path.join(os.getcwd(), 'zhang_anomalies')
_, _, filenames = next(os.walk(data_dir))
print("Testing Portfolio files")
print('Show the first 5...')
print(filenames[0:5])

for three_sort in range(1,4):

    for five_sort in range(1,6):

        anomalies = pd.DataFrame()

        for fil in filenames:
            file_path = os.path.join(data_dir, fil)
            df = pd.read_csv(file_path)
            df["month"] = df["month"].astype(str)
            for i in range(9):
                df.loc[df["month"] == str(i), 'month'] = "0" + df.loc[df["month"] == str(i), 'month']
            df.loc[:, 'ret_vw'] = df.loc[:, 'ret_vw'] / 100.0
            df['date'] = 0.0
            df['date'] = df["year"].astype(str) + df["month"].astype(str) 
            df.index = pd.to_datetime(df.date, format= '%Y%m')

            three_sort_name = ''
            five_sort_name = ''
            col_names = df.columns
            for i in col_names:
                if 'rank' in i and 5 in df[i].unique():
                    five_sort_name = i
                if 'rank' in i and 5 not in df[i].unique():
                    three_sort_name = i
            df_add = df.loc[df[three_sort_name] == three_sort]
            df_add = df_add.loc[df_add[five_sort_name] == five_sort]
            df_add = pd.DataFrame(df_add['ret_vw'])
            df_add.columns = [fil.replace('_monthly_2019a.csv', '')]
            if anomalies.empty:
                anomalies = df_add.copy()
            else:
                anomalies = pd.concat([anomalies, df_add], axis = 1)
                
        ff_factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', index_col = 0)
        ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')
        ff_factors = ff_factors / 100.0
        ff_factors[ff_factors == -999] = np.NaN
        ff_factors[ff_factors == -99.9] = np.NaN
        ff_factors = ff_factors.dropna(how = 'any', axis = 0)
        ff_factors = ff_factors.dropna(axis = 0, how = 'any')
        ff_factors = ff_factors.loc[anomalies.index].dropna(axis = 0)
        
        factor_model = ff_factors.copy().dropna(axis = 0, how = 'any')
        rf = factor_model[['RF']]
        factor_model = factor_model[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
        print(factor_model)
        print(rf)

        ff_factors_compare = factor_model.copy()
        
        anomalies = anomalies.loc[:, anomalies.columns[0:30]]
        #start date for data
        start = '2004-01-01'
        #end date for data
        end = '2020-01-01'

        #the lookback for alpha feature
        lookback = 60
        #control the leverage
        lev = 1.
        stk_symbols  = anomalies.columns
        returns = anomalies.copy()


        #set the backtest parameters
        bp = BacktestParameters(stk_symbols, start, end, lookback, lev  )
        bt = Backtest(bp, returns, mode = 'insample')
        data = Data(bp)

        df_alpha, idiosyncratic_vol, binary_alpha = data.get_alpha(returns, factor_model, rf)
        rolling_mean_12, rolling_mean_6, rolling_std_12, binary_mean_12,binary_mean_6,binary_returns = data.get_rolling_features(returns, 0.0)


        #creates a dictionary of different features for analysis based on ticker keys
        feature_list = [binary_alpha]
        #feature_list = [df_alpha]

        feature_set = data.get_features(feature_list)

        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')

        print(backtest)
        
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)

        output_reg = []
        output_capm = []
        names = ['KNN L/S', 'KNN L', 'KNN S', 'Neural Net L/S', 'Neural Net L', 'Neural Net S', 'RF L/S', 'RF L', 'RF S', 'Ensemble', 'Binary']


        print("KNN")
        print("Long and Short")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Long Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Short Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Neural Net")
        print("Long and Short")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Long Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Short Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Random Forest")
        print("Long and Short")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Long Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Short Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Ensemble System")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')  
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()

        print("Simple Binary Decision")
        df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
        backtest = returns.loc[df_alpha_shift.index]
        weights = returns.loc[df_alpha_shift.index]
        for count, row in  backtest.iterrows(): 
            for j in backtest.columns:
                if df_alpha_shift.loc[count, j] >= 0:
                    backtest.loc[count, j] = backtest.loc[count, j]
                    weights.loc[count, j] = 1.0
                else:
                    backtest.loc[count, j] = -backtest.loc[count, j]
                    weights.loc[count, j] = -1.0

        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()


        print(summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names))


        tp_factor_results = summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names)
        tp_factor_results.title = 'Testing Portfolios- Hou, Kewei, Chen Xue, and Lu Zhang' + ' ' + str(three_sort) + ' ' + str(five_sort)
        tp_factor_results_capm = summary_col(output_capm,stars=True,float_format='%0.4f',model_names=names)
        tp_factor_results_capm.title = 'Testing Portfolios- Hou, Kewei, Chen Xue, and Lu Zhang' + ' ' + str(three_sort) + ' ' + str(five_sort) + ' ' + 'capm'
        resultFile = open("output.txt",'a')
        resultFile.write(m4_factor_results.as_text())
        resultFile.write('\n')
        resultFile.write(m4_factor_results_capm.as_text())
        resultFile.write('\n')
        resultFile.close()




Testing Portfolio files
Show the first 5...
['portf_me_dbe_monthly_2019a.csv', 'portf_me_sim_1_monthly_2019a.csv', 'portf_me_ivg_monthly_2019a.csv', 'portf_me_eg_6_monthly_2019a.csv', 'portf_me_abr_1_monthly_2019a(1).csv']
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]
                RF
date              
1967-01-01  0.0043
1967-02-01  0.0036
1967-03-01  0.

Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.001106   0.002246  0.492517
Mkt-RF     0.000054   0.000181  0.298393
SMB        0.000093   0.000125  0.738973
HML       -0.000090   0.000095 -0.944444
RMW       -0.000071   0.000087 -0.825353
CMA        0.000036   0.000061  0.588658
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.003601   0.002109  1.707511
Mkt-RF     0.000258   0.000168  1.538528
SMB        0.000168   0.000110  1.532624
HML       -0.000026   0.000082 -0.319290
RMW       -0.000094   0.000072 -1.306256
CMA        0.000021   0.000052  0.395972
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.003863   0.002117  1.824696
Mkt-RF     0.000305   0.000171  1.787890
SMB        0.000179   0.000116  1.537274
HML       -0.000050   0.000084 -0.596444
RMW       -0.000097   0.000077 -1.261227
CMA        0.000006   0.000050  0.112461
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.003401   0.002108  1.613814
Mkt-RF     0.000250   0.000164  1.521621
SMB        0.000147   0.000111  1.324779
HML       -0.000020   0.000083 -0.235769
RMW       -0.000082   0.000078 -1.050786
CMA        0.000009   0.000051  0.175775
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


...

Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.004173   0.001483  2.814841
Mkt-RF     0.000407   0.000148  2.753521
SMB       -0.000010   0.000066 -0.148131
HML       -0.000075   0.000061 -1.230644
RMW       -0.000008   0.000041 -0.189045
CMA       -0.000035   0.000039 -0.883242
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.003920   0.001444  2.713971
Mkt-RF     0.000508   0.000145  3.490935
SMB       -0.000011   0.000066 -0.166081
HML       -0.000080   0.000054 -1.488461
RMW        0.000009   0.000034  0.259305
CMA       -0.000056   0.000032 -1.780121
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.004301   0.001438  2.991248
Mkt-RF     0.000505   0.000145  3.490424
SMB       -0.000029   0.000065 -0.441046
HML       -0.000085   0.000053 -1.593706
RMW        0.000002   0.000034  0.046174
CMA       -0.000048   0.000032 -1.480108
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.004179   0.001423  2.936940
Mkt-RF     0.000566   0.000142  3.980014
SMB        0.000007   0.000063  0.104642
HML       -0.000105   0.000052 -2.005038
RMW       -0.000002   0.000032 -0.062928
CMA       -0.000059   0.000032 -1.821516
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
...            ...     ...     ...     ...     ...
2019-08-01 -0.0258 -0.0330 -0.0493  0.0043 -0.0088
2019-09-01  0.0143  0.0026  0.0678  0.0199  0.0350
2019-10-01  0.0206  0.0021 -0.0209  0.0025 -0.0099
2019-11-01  0.0387  0.0050 -0.0187 -0.0158 -0.0124
2019-12-01  0.0277  0.0096  0.0193  0.0009  0.0129

[636 rows x 5 columns]


Second Stage Fama Macbeth
               mean  std_error     tstat
intercept  0.003908   0.001541  2.535466
Mkt-RF     0.000542   0.000156  3.482673
SMB        0.000043   0.000068  0.635188
HML       -0.000162   0.000064 -2.536977
RMW       -0.000019   0.000043 -0.450663
CMA       -0.000090   0.000041 -2.216216


In [ ]:
output_reg = []
output_capm = []
names = ['KNN L/S', 'KNN L', 'KNN S', 'Neural Net L/S', 'Neural Net L', 'Neural Net S', 'RF L/S', 'RF L', 'RF S', 'Ensemble', 'Binary']


print("KNN")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Neural Net")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Random Forest")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()

print("Ensemble System")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')  
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
    
print("Simple Binary Decision")
df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
backtest = returns.loc[df_alpha_shift.index]
weights = returns.loc[df_alpha_shift.index]
for count, row in  backtest.iterrows(): 
    for j in backtest.columns:
        if df_alpha_shift.loc[count, j] >= 0:
            backtest.loc[count, j] = backtest.loc[count, j]
            weights.loc[count, j] = 1.0
        else:
            backtest.loc[count, j] = -backtest.loc[count, j]
            weights.loc[count, j] = -1.0
            
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()


print(summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names))


tp_factor_results = summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names)
tp_factor_results.title = 'Testing Portfolios- Hou, Kewei, Chen Xue, and Lu Zhang' + ' ' + str(three_sort) + ' ' + str(five_sort)
tp_factor_results_capm = summary_col(output_capm,stars=True,float_format='%0.4f',model_names=names)
tp_factor_results_capm.title = 'Testing Portfolios- Hou, Kewei, Chen Xue, and Lu Zhang' + ' ' + str(three_sort) + ' ' + str(five_sort) + ' ' + 'capm'
resultFile = open("output.txt",'a')
resultFile.write(m4_factor_results.as_text())
resultFile.write('\n')
resultFile.write(m4_factor_results_capm.as_text())
resultFile.write('\n')
resultFile.close()

